In [5]:
import gensim as gs
from gensim import corpora as cor
import os
import json
from pandas import to_datetime
from text_pipeline import *

In [6]:
os.chdir('/home/matt/Desktop/sandbox/DTM_test/')

In [7]:
dtmpath = 'dtm/dtm/main'
processorconfigpath = 'complete_unigrams_config.json'
docdir='data'

In [8]:
class RecordIter:
    #iterates over individual records in a file, in this case a json with records in a slot labelled 'Reviews'
    def __init__(self, filepath, encodings=['iso-8859-1','utf-16le']):
        with open(filepath,'r') as readfile:
            data = json.load(readfile)
            self.data = data['Reviews']
            self.docs = iter(self.data)
            
    def __iter__(self):
        return self
    
    def __next__(self):
        try:
            nextdoc = next(self.docs)
        except:
            raise StopIteration
        else:
            return nextdoc


In [9]:
docs = DocIter(docdir,recursive=True,recordIter=RecordIter)
processor = Processor(processorconfigpath)
id2word = cor.Dictionary()

In [10]:
corpus = []
months = []
years = []

In [11]:
for doc in docs:
    
    #print(doc)
    #doc = docs.__next__()
    
    try:
        date = to_datetime(doc['Date'])
        #print(date)
    except:
        continue
    else:
        months.append(date.month)
        years.append(date.year)
        
        tokens = gs.utils.simple_preprocess(doc['Content'])
        #tokens = [str(token) for token in tokens]
        id2word.add_documents([tokens])
        bow = id2word.doc2bow(tokens)
        corpus.append(bow)

In [12]:
# class gensim.models.dtmmodel.DtmModel(dtm_path, corpus=None, time_slices=None, 
#                                       num_topics=100, id2word=None, prefix=None, 
#                                       lda_sequence_min_iter=6, lda_sequence_max_iter=20, 
#                                       lda_max_em_iter=10, alpha=0.01, top_chain_var=0.005, 
#                                       rng_seed=0, initialize_lda=False)
#
#    Bases: gensim.utils.SaveLoad
#
#    Class for DTM training using DTM binary. Communication between DTM and Python takes place by passing around data files on disk and executing the DTM binary as a subprocess.
#
#    dtm_path is path to the dtm executable, e.g. C:/dtm/dtm-win64.exe.
#
#    corpus is a gensim corpus, aka a stream of sparse document vectors.
#
#    id2word is a mapping between tokens ids and token.
#
#    lda_sequence_min_iter min iteration of LDA.
#
#    lda_sequence_max_iter max iteration of LDA.
#
#    lda_max_em_iter max em optiimzatiion iterations in LDA.
#
#    alpha is a hyperparameter that affects sparsity of the document-topics for the LDA models in each timeslice.
#
#    top_chain_var is a hyperparameter that affects.
#
#    rng_seed is the random seed.
#
#    initialize_lda initialize DTM with LDA.

In [13]:
# So this DTM model has this idiotic way of representing timeslices as # docs in each slice; the 
# docs have to be SORTED by time first.
# So I made up some timeslices here following that format rather than sort for now
timeslices = [(len(corpus)//11)]*11

timeslices[10] = len(corpus) - sum(timeslices[0:10])

In [14]:
DTM = gs.models.wrappers.DtmModel(dtm_path=dtmpath,corpus=corpus,id2word=id2word,time_slices=timeslices,num_topics=40)

CalledProcessError: Command '['dtm/dtm/main', '--ntopics=40', '--model=dtm', '--mode=fit', '--initialize_lda=false', '--corpus_prefix=/tmp/76c91d_train', '--outname=/tmp/76c91d_train_out', '--alpha=0.01', '--lda_max_em_iter=10', '--lda_sequence_min_iter=6', '--lda_sequence_max_iter=20', '--top_chain_var=0.005', '--rng_seed=0']' returned non-zero exit status -11